<a href="https://colab.research.google.com/github/XiuqiXi/colabNotebook/blob/main/ManimExample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt update
!sudo apt install libcairo2-dev ffmpeg \
    texlive texlive-latex-extra texlive-fonts-extra \
    texlive-latex-recommended texlive-science \
    tipa libpango1.0-dev
!pip install manim
!pip install IPython --upgrade

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [903 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelea

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 793 kB 4.9 MB/s 
     |████████████████████████████████| 381 kB 52.2 MB/s 
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18
  Attempting uninstall: IPython
    Found existing installation: ipython 5.5.0
    Uninstalling ipython-5.5.0:
      Successfully uninstalled ipython-5.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.30 which is incompatible.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.34.0 which is incompatible.


In [ ]:
from manim import *

Manim Community v0.16.0.post0

In [ ]:
%%manim -qm -v WARNING SquareToCircle

class SquareToCircle(Scene):
   def construct(self):
      square = Square()
      circle = Circle()
      circle.set_fill(RED, opacity=0.9)
      self.play(Create(circle))
      self.play(Transform(circle, square))
      self.wait()

UsageError: Cell magic `%%manim` not found.


In [ ]:
%%manim -qm -v WARNING SquareAndCircle

class SquareAndCircle(Scene):
    def construct(self):
        circle = Circle()  # create a circle
        circle.set_fill(PINK, opacity=0.5)  # set the color and transparency

        square = Square()  # create a square
        square.set_fill(BLUE, opacity=0.5)  # set the color and transparency

        square.next_to(circle, UP, buff=-1)  # set the position
        self.play(Create(circle), Create(square))  # show the shapes on screen

UsageError: Cell magic `%%manim` not found.


In [ ]:
%%manim -qm -v WARNING MathTeXDemo

class MathTeXDemo(Scene):
    def construct(self):
        rtarrow0 = MathTex(r"B_{R}=\frac{(1-\chi) \Delta H Y_{\infty} s-c_{p g}\left(T_{i g}-T_{\infty}\right)}{L+\left(\dot{q}_{f, r}^{\prime \prime}-\dot{q}_{s, r}^{\prime \prime}-\dot{q}_{s, c}^{\prime \prime}\right) / \dot{m}_{\mathrm{f}}^{\prime \prime}}", font_size=40)
        rtarrow1 = Tex(r"$B=\frac{\Delta H Y_{\infty \infty} s-c_{p g}\left(T_{\text {ig }}-T_{\infty}\right)}{L+Q}$", font_size=40)

        rtarrow1.next_to(rtarrow0, UP, buff=1.5)  # set the position

        self.play(Create(rtarrow0), Create(rtarrow1))

        self.play(Transform(rtarrow1, rtarrow0))

        self.wait()


In [ ]:
%%manim -qm -v WARNING CreateGraph


class CreateGraph(Scene):
    def construct(self):
        axes = Axes(
            x_range=[0, 12],
            y_range=[0, 4],
            axis_config={"color": BLUE},
        )

        # Create Graph
        graph = axes.plot(lambda x: x**0.5, color=WHITE)

        axes_1 = Axes(
            x_range=[2, 12],
            y_range=[0, 4],
            axis_config={"color": BLUE},
        )

        # Create Graph
        graph_1 = axes_1.plot(lambda x: (x-2)**2, color=WHITE)

        

        # Display graph
        self.play(Create(axes), Create(graph))
        self.wait(1)
        self.play(Create(graph_1))
        self.wait(1)


In [ ]:
%%manim -qm -v WARNING SeveralArcPolygons

class SeveralArcPolygons(Scene):
    def construct(self):
      axes = Axes(
          x_range=[0, 24],
          y_range=[0, 15],
          axis_config={"color": BLUE},
      )

      # Create Graph
      graph = axes.plot(lambda x: 2*x**0.5, color=WHITE)


      #bezier1 = CubicBezier([-2,-3,0], [1,-1,0],  [-1.8,1,0],  [-2,3,0])

      bezier2 = CubicBezier([-2,-3,0], [0,-1,0],  [-2,3,0],  [-2,3,0])

      arc= ArcBetweenPoints([-2,-3,0], [2,1,0], stroke_color=YELLOW, angle=2)

      self.play(Create(axes), Create(graph))
      #self.wait(1)

      self.play(Create(bezier2))
      #self.play(Create(bezier2))
      self.wait(1)

In [ ]:
%%manim -qm -v WARNING boundaryFlow

class boundaryFlow(Scene):
    def construct(self):
        rtarrow0 = Tex(r"$\frac{\partial(\rho u)}{\partial x}+\frac{\partial(\rho v)}{\partial y}=0$", font_size=40)
        rtarrow1 = Tex(r"$\rho\left(u \frac{\partial u}{\partial x}+v \frac{\partial u}{\partial y}\right)=-\frac{\partial p}{\partial x}+\mu\left(\frac{\partial^{2} u}{\partial x^{2}}+\frac{\partial^{2} u}{\partial y^{2}}\right)$", font_size=40)
        rtarrow2 = Tex(r"$\rho\left(u \frac{\partial v}{\partial x}+v \frac{\partial v}{\partial y}\right)=-\frac{\partial p}{\partial y}+\mu\left(\frac{\partial^{2} v}{\partial x^{2}}+\frac{\partial^{2} v}{\partial y^{2}}\right)$", font_size=40)

        rtarrow2.next_to(rtarrow1, DOWN, buff=1.5)  # set the position

        rtarrow0.next_to(rtarrow1, UP, buff=1.5)  # set the position

        self.play(Create(rtarrow0), Create(rtarrow1), Create(rtarrow2))

        rtarrow3 = Tex(r"$\rho\left(u \frac{\partial u}{\partial x}+v \frac{\partial u}{\partial y}\right)=\mu\frac{\partial^{2} u}{\partial y^{2}}$", font_size=40)

        #rtarrow3.next_to(rtarrow1, DOWN, buff=1.5)  # set the position

        self.play(AnimationGroup(FadeOut(rtarrow1), lag_ratio=0.5))

        self.play(Create(rtarrow3))

        self.wait()

In [ ]:
%%manim -qm -v WARNING OpeningManim


class OpeningManim(Scene):
    def construct(self):
        title = Tex(r"This is some \LaTeX")
        basel = MathTex(r"\sum_{n=1}^\infty \frac{1}{n^2} = \frac{\pi^2}{6}")
        VGroup(title, basel).arrange(DOWN)
        self.play(
            Write(title),
            FadeIn(basel, shift=DOWN),
        )
        self.wait()

        transform_title = Tex("That was a transform")
        transform_title.to_corner(UP + LEFT)
        self.play(
            Transform(title, transform_title),
            LaggedStart(*[FadeOut(obj, shift=DOWN) for obj in basel]),
        )
        self.wait()

        grid = NumberPlane()
        grid_title = Tex("This is a grid", font_size=72)
        grid_title.move_to(transform_title)

        self.add(grid, grid_title)  # Make sure title is on top of grid
        self.play(
            FadeOut(title),
            FadeIn(grid_title, shift=UP),
            Create(grid, run_time=3, lag_ratio=0.1),
        )
        self.wait()

        grid_transform_title = Tex(
            r"That was a non-linear function \\ applied to the grid"
        )
        grid_transform_title.move_to(grid_title, UL)
        grid.prepare_for_nonlinear_transform()
        self.play(
            grid.animate.apply_function(
                lambda p: p
                          + np.array(
                    [
                        np.cos(p[1]),
                        np.sin(p[0]),
                        0,
                    ]
                )
            ),
            run_time=3,
        )
        self.wait()
        self.play(Transform(grid_title, grid_transform_title))
        self.wait()

In [ ]:
import numpy as np

# Original Function
func_period_from, func_period_to = -PI, PI
func_discontinuities = np.arange(-PI * 4, PI * 4, PI)


def func(x):
    return 0 if x >= 0 else 1


# Fourier series for the func above
def series(x, num_terms):
    sum_terms = 1 / 2

    # reverse order
    # adding up from small numbers
    for n in range(num_terms, 0, -1):
        if (n % 2) == 0:
            continue
        sum_terms += -(2 / (n * PI)) * np.sin(n * x)

    return sum_terms


# Tex
# Web Tool : Hand writing math expression to Tex
#   https://webdemo.myscript.com/views/math/index.html#
tex_func = r"""
f(x) =
\begin{cases}
    \hspace{.4em}
    1 \quad ( \, -\pi \leq x < 0 \, ) \\
    \hspace{.4em}
    0 \quad ( \, 0 \leq x < \pi \, )
\end{cases}
"""
tex_sim_or_equals = r"\sim"
tex_series = r"""
\dfrac{1}{2} +
\sum_{n=1}^{\infty}
    \dfrac{1}{n\pi}
    \Big\{ (-1) ^{n}-1 \Big\} \sin nx
"""

# Number of terms
series_start, series_end, series_step = 1, 19, 2
series_run_time = 15

series_more_start, series_more_end, series_more_step = 21, 201, 10
series_more_run_time = 10

# Axes
x_min, x_max, x_step = -3.4 * PI, 3.4 * PI, PI
y_min, y_max, y_step = -0.1, 1.5, 1
y_length = 3
include_ticks = False
include_tip = False
is_pi_label = True
show_x_0_label = True

# Function plot
func_x_range_min, func_x_range_max = -3.3 * PI, 3.3 * PI
plot_x_delta = 5e-3

# Color
COLOR_MESSAGE = WHITE
COLOR_FUNC = GREEN
COLOR_PERIODIC = YELLOW
COLOR_SERIES = BLUE
COLOR_TILDE = WHITE
COLOR_N_TERM_COUNT = WHITE

# Message displayed before the formula
message = "Fourier Series for"

In [ ]:
%%manim -qm -v WARNING Fourier


def periodic(offset, period):
    return lambda f: lambda x: f(((x - offset) % period) + offset)


func_period = func_period_to - func_period_from


@periodic(func_period_from, func_period)
def func_periodic(x):
    return func(x)


def func_series(n):
    @periodic(func_period_from, func_period)
    def func_series_periodic(x):
        return series(x, n)
    return func_series_periodic


class Fourier(Scene):
    def construct(self):
        self.create_axes()

        # define the original function
        graph_func = self.axes.plot(
            func_periodic,
            x_range=(func_x_range_min, func_x_range_max, plot_x_delta),
            discontinuities=func_discontinuities,
            use_smoothing=True,
            color=COLOR_FUNC
        )

        # message displayed before the formula
        txt_message = Text(message, font_size=40)
        txt_message.set_color(COLOR_MESSAGE)

        math_func = MathTex(tex_func, font_size=40)
        math_func.set_color(COLOR_FUNC)

        math_tilde = MathTex(tex_sim_or_equals, font_size=60)
        math_tilde.set_color(COLOR_TILDE)

        math_series = MathTex(tex_series, font_size=40)
        math_series.set_color(COLOR_SERIES)

        top_labels = VGroup(txt_message, math_tilde, math_series).arrange(RIGHT, buff=1)
        top_labels.to_edge(UP, buff=1)
        math_func.move_to(txt_message)

        txt_periodic = Text("periodic", font_size=22, color=COLOR_PERIODIC).next_to(math_func, DOWN, buff=0.2)

        # Animation Tutorial
        #     https://azarzadavila-manim.readthedocs.io/en/latest/animation.html
        # Animation Transform Src
        #     https://github.com/ManimCommunity/manim/blob/main/manim/animation/transform.py
        self.play(Write(txt_message), DrawBorderThenFill(self.axes))
        self.play(ReplacementTransform(txt_message, math_func), Write(txt_periodic), Create(graph_func))
        self.play(Write(math_series))
        self.play(Write(math_tilde))

        self.prepare_fourier_animation()
        self.animate_fourier_series(series_start, series_end, series_step)
        self.animate_fourier_series(series_more_start, series_more_end, series_more_step)

        self.wait()

    def prepare_fourier_animation(self):
        self.math_n_equals = MathTex(r"n = ", font_size=60, color=COLOR_N_TERM_COUNT)
        self.math_n_equals.align_to(self.axes, UR).shift(LEFT)
        self.play(FadeIn(self.math_n_equals))

        self.graph_series = None
        self.n_series = None

    def animate_fourier_series(self, start, end, step):
        each_run_time = series_run_time / (end - start)

        fade_animation_on = each_run_time >= 0.4

        # transform the graph while increasing the number of terms
        for n in range(start, end + 1, step):
            # MEMO : I could use Variable class too.
            # Variable class = ValueTracker with label
            #     https://docs.manim.community/en/stable/reference/manim.mobject.text.numbers.Variable.html

            n_series_new = Integer(n).scale(1.1).next_to(self.math_n_equals, RIGHT, buff=0.2).shift(UP * 0.03)
            n_series_new.set_color(COLOR_N_TERM_COUNT)

            # ParametricFunction
            #     https://docs.manim.community/en/stable/reference/manim.mobject.graphing.functions.ParametricFunction.html
            graph_series_new = self.axes.plot(
                func_series(n),
                x_range=[func_x_range_min, func_x_range_max, plot_x_delta],
                use_smoothing=True, color=COLOR_SERIES, stroke_width=3, stroke_opacity=0.9)
            if self.graph_series is not None:
                self.play(
                    ReplacementTransform(self.graph_series, graph_series_new, run_time=each_run_time),
                    FadeTransform(self.n_series, n_series_new, run_time=each_run_time if fade_animation_on else 0))
            else:
                self.play(Create(graph_series_new), Write(n_series_new))

            self.graph_series = graph_series_new
            self.n_series = n_series_new

    def create_axes(self):
        self.axes = (Axes(
            x_range=(x_min, x_max, x_step),
            y_range=(y_min, y_max, y_step), y_length=y_length,
            # axis_config keys available
            #   https://docs.manim.community/en/stable/reference/manim.mobject.graphing.number_line.NumberLine.html#manim.mobject.graphing.number_line.NumberLine
            axis_config={
                "include_ticks": include_ticks,
                "include_tip": include_tip
            }
        )).to_corner(DOWN, buff=1.8)

        labels = self.axes.get_axis_labels(x_label="x", y_label="").set_color(WHITE)
        self.axes.add(labels)

        if(is_pi_label):
            self.add_pi_x_labels()

    def add_pi_x_labels(self, scale=0.8, buff=0.4, buff_adjust=0.09, aligned_edge=DOWN):
        x_min = self.axes.x_range[0]
        x_max = self.axes.x_range[1]

        if show_x_0_label and x_min <= 0 and 0 <= x_max:
            self.axes.add(
                MathTex(r"0").next_to(
                    self.axes.c2p(0, 0),
                    aligned_edge, buff=buff
                ).scale(scale)
            )
        if x_min <= -PI and -PI <= x_max:
            self.axes.add(
                MathTex(r"-\pi").next_to(
                    self.axes.c2p(-PI, 0),
                    aligned_edge,
                    buff=buff + buff_adjust
                ).scale(scale)
            )
        if x_min <= PI and PI <= x_max:
            self.axes.add(
                MathTex(r"\pi").next_to(
                    self.axes.c2p(PI, 0),
                    aligned_edge,
                    buff=buff + buff_adjust
                ).scale(scale)
            )

        for n in range(-10, 10):
            if n in (-1, 0, 1):
                continue
            npi = n * PI
            if x_min <= npi and npi <= x_max:
                self.axes.add(
                    MathTex(str(n) + r"\pi").next_to(
                        self.axes.c2p(npi, 0),
                        aligned_edge,
                        buff=buff
                    ).scale(scale)
                )

Animation 15: ReplacementTransform(ParametricFunction), etc.:   0%|          | 0/3 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/manim/scene/scene.py:1427: RuntimeWarning: invalid value encountered in double_scalars
  alpha = t / animation.run_time
/usr/local/lib/python3.7/dist-packages/manim/scene/scene.py:1427: RuntimeWarning: divide by zero encountered in double_scalars
  alpha = t / animation.run_time
Animation 16: ReplacementTransform(ParametricFunction), etc.:   0%|          | 0/3 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/manim/scene/scene.py:1427: RuntimeWarning: invalid value encountered in double_scalars
  alpha = t / animation.run_time
/usr/local/lib/python3.7/dist-packages/manim/scene/scene.py:1427: RuntimeWarning: divide by zero encountered in double_scalars
  alpha = t / animation.run_time
Animation 19: ReplacementTransform(ParametricFunction), etc.:   0%|          | 0/3 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/manim/scene/scene.py:1427: